# Поведение реализаций эвристического алгоритма

На нескольких примерах анализируются результаты работы трёх реализаций эвристического алгоритма: `Heuristicterations()`, `Heuristicterations!()` и `RecursiveIteration!()`.

> Исправлены ошибки в реализации функции `OptimizeRanking!()`. Во-первых,
для перестановки элементов в векторе альтернатив стоит проверять не очередные
значения в _преобразованной_ матрице потерь (`R[k,k+1] > R[k+1,k]`), а значения, определяемые
текущим ранжированием альтернатив (`R[P[k],P[k+1]] > R[P[k+1],P[k]]`), тогда вторым
параметром функции можно будет передавать не преобразованную, а исходную матрицу,
во-вторых, при перестановке надо делать дополнительный «шаг» (`k -= 1`), чтобы
избежать нарушения только что переставленной пары.

> Эти изменения позволили добиться здравых результатов при переходе к истинной
медиане Кемени от получаемых в процессе итераций эвристического алгоритма
приближений к ней. При этом оказалось, что «оптимизация» последовательности --
не одношаговая процедура, её имеет смысл повторять снова и снова -- вплоть
до получения «стабильной» последовательности, не изменяющейся далее в результате
«оптимизации».

> Теперь (для наглядности) получаемая рекурсивным вариантом последовательность (ранжирование)
сопровождается вычисляемой суммой наддиагональных элементов матрицы потерь при
применении к ней перестановок строк/стобцов в соответствии с ранжированием,
что позволяет убедиться, что шаги «оптимизации» происходят в «правильном» направлении.

In [1]:
include("RecursiveIterations.jl")

RecursiveIteration! (generic function with 2 methods)

## Пример 1

«Облегчённая» матрица потерь для примера, часто используемого при иллюстрации парадокса Кондорсе.

In [2]:
LCP = [0 1 -1; -1 0 1; 1 -1 0]

3×3 Array{Int32,2}:
  0   1  -1
 -1   0   1
  1  -1   0

In [3]:
print(HeuristicIterations(LCP))

[1, 3, 2]

In [4]:
ReOrder(LCP,[1,3,2])

3×3 Array{Int32,2}:
  0  -1   1
  1   0  -1
 -1   1   0

In [5]:
print(OptimizeRanking!([1,3,2],LCP))

[1, 3, 2]

Традиционная реализация эвристического алгоритма находит одно из оптимальных ранжирований, рекурсивный вариант обнаруживает все три.

In [6]:
R = copy(LCP); RecursiveIteration!(R)

[1, 3, 2] : -1

[2, 1, 3] : -1

[3, 2, 1] : -1



## Пример 2

Пример из книги Б.Литвака (стр.83).

In [7]:
LL83 = [0 2 0 1; -2 0 1 -2; 0 -1 0 -1; -1 2 1 0]

4×4 Array{Int32,2}:
  0   2  0   1
 -2   0  1  -2
  0  -1  0  -1
 -1   2  1   0

In [8]:
print(HeuristicIterations(LL83))

[2, 3, 4, 1]

In [9]:
ReOrder(LL83,[2, 3, 4, 1])

4×4 Array{Int32,2}:
  0  1  -2  -2
 -1  0  -1   0
  2  1   0  -1
  2  0   1   0

In [10]:
print(OptimizeRanking!([2, 3, 4, 1],LL83))

[3, 2, 4, 1]

In [11]:
R = copy(LL83); RecursiveIteration!(R)

[2, 3, 4, 1] : -5 -> [3, 2, 4, 1] : -7



In [12]:
ReOrder(LL83,[3, 2, 4, 1])

4×4 Array{Int32,2}:
 0  -1  -1   0
 1   0  -2  -2
 1   2   0  -1
 0   2   1   0

## Пример 3

Пример «облегчённой» матрицы потерь, для которой не находится оптимальное ранжирование (даже после оптимизационного шага).

In [13]:
LMT = [0 -3 -3 1 -1 -3 -1 -3 -1 -3; 3 0 5 3 3 1 1 5 1 1; 3 -5 0 3 -1 -1 1 -1 -3 -3; -1 -3 -3 0 -3 -1 1 -3 -3 -3; 1 -3 1 3 0 1 1 1 -1 -1; 3 -1 1 1 -1 0 1 1 -1 -1; 1 -1 -1 -1 -1 -1 0 -1 1 -1; 3 -5 1 3 -1 -1 1 0 -1 -1; 1 -1 3 3 1 1 -1 1 0 1; 3 -1 3 3 1 1 1 1 -1 0]

10×10 Array{Int32,2}:
  0  -3  -3   1  -1  -3  -1  -3  -1  -3
  3   0   5   3   3   1   1   5   1   1
  3  -5   0   3  -1  -1   1  -1  -3  -3
 -1  -3  -3   0  -3  -1   1  -3  -3  -3
  1  -3   1   3   0   1   1   1  -1  -1
  3  -1   1   1  -1   0   1   1  -1  -1
  1  -1  -1  -1  -1  -1   0  -1   1  -1
  3  -5   1   3  -1  -1   1   0  -1  -1
  1  -1   3   3   1   1  -1   1   0   1
  3  -1   3   3   1   1   1   1  -1   0

In [14]:
print(HeuristicIterations(LMT))

[4, 1, 3, 8, 5, 6, 7, 10, 9, 2]

In [15]:
ReOrder(LMT,[4, 1, 3, 8, 5, 6, 7, 10, 9, 2])

10×10 Array{Int32,2}:
  0  -1  -3  -3  -3  -1   1  -3  -3  -3
  1   0  -3  -3  -1  -3  -1  -3  -1  -3
  3   3   0  -1  -1  -1   1  -3  -3  -5
  3   3   1   0  -1  -1   1  -1  -1  -5
  3   1   1   1   0   1   1  -1  -1  -3
  1   3   1   1  -1   0   1  -1  -1  -1
 -1   1  -1  -1  -1  -1   0  -1   1  -1
  3   3   3   1   1   1   1   0  -1  -1
  3   1   3   1   1   1  -1   1   0  -1
  3   3   5   5   3   1   1   1   1   0

In [16]:
UpperSum(ReOrder(LMT,[4, 1, 3, 8, 5, 6, 7, 10, 9, 2]))

-67

In [17]:
print(OptimizeRanking!([4, 1, 3, 8, 5, 6, 7, 10, 9, 2],LMT))

[4, 1, 3, 8, 5, 7, 6, 10, 9, 2]

In [18]:
UpperSum(ReOrder(LMT,[4, 1, 3, 8, 5, 7, 6, 10, 9, 2]))

-69

Рекурсивный вариант возвращает несколько ранжирований, но не все из них оптимизируются. Те, что достигли оптимального, просто совпадают.

In [19]:
R = copy(LMT); RecursiveIteration!(R)

[4, 1, 3, 8, 5, 6, 7, 10, 9, 2] : -67 -> [4, 1, 3, 8, 5, 7, 6, 10, 9, 2] : -69 -> [4, 1, 3, 8, 7, 5, 6, 10, 9, 2] : -71 -> [4, 1, 3, 7, 8, 6, 5, 10, 9, 2] : -75 -> [4, 1, 7, 3, 8, 6, 5, 10, 9, 2] : -77

[4, 1, 3, 8, 5, 6, 9, 7, 10, 2] : -67 -> [4, 1, 3, 8, 6, 5, 9, 7, 10, 2] : -69

[4, 1, 3, 8, 5, 6, 10, 9, 7, 2] : -67 -> [4, 1, 3, 8, 6, 5, 10, 9, 7, 2] : -69

[4, 1, 3, 8, 5, 7, 6, 10, 9, 2] : -69 -> [4, 1, 3, 8, 7, 5, 6, 10, 9, 2] : -71 -> [4, 1, 3, 7, 8, 6, 5, 10, 9, 2] : -75 -> [4, 1, 7, 3, 8, 6, 5, 10, 9, 2] : -77

[4, 1, 3, 8, 6, 5, 7, 10, 9, 2] : -69 -> [4, 1, 3, 8, 6, 7, 5, 10, 9, 2] : -71 -> [4, 1, 3, 8, 7, 6, 5, 10, 9, 2] : -73 -> [4, 1, 3, 7, 8, 6, 5, 10, 9, 2] : -75 -> [4, 1, 7, 3, 8, 6, 5, 10, 9, 2] : -77

[4, 1, 3, 8, 6, 5, 9, 7, 10, 2] : -69

[4, 1, 3, 8, 6, 5, 10, 9, 7, 2] : -69

[4, 1, 3, 8, 7, 5, 6, 10, 9, 2] : -71 -> [4, 1, 3, 7, 8, 6, 5, 10, 9, 2] : -75 -> [4, 1, 7, 3, 8, 6, 5, 10, 9, 2] : -77

[4, 1, 3, 8, 7, 6, 5, 10, 9, 2] : -73 -> [4, 1, 3, 7, 8, 6, 5, 10, 9, 2] 

Оптимальное ранжирование оставляет в нижнем треугольнике всего два отрицательных значения:

In [20]:
ReOrder(LMT,[4, 1, 7, 3, 8, 6, 5, 10, 9, 2])

10×10 Array{Int32,2}:
  0  -1   1  -3  -3  -1  -3  -3  -3  -3
  1   0  -1  -3  -3  -3  -1  -3  -1  -3
 -1   1   0  -1  -1  -1  -1  -1   1  -1
  3   3   1   0  -1  -1  -1  -3  -3  -5
  3   3   1   1   0  -1  -1  -1  -1  -5
  1   3   1   1   1   0  -1  -1  -1  -1
  3   1   1   1   1   1   0  -1  -1  -3
  3   3   1   3   1   1   1   0  -1  -1
  3   1  -1   3   1   1   1   1   0  -1
  3   3   1   5   5   1   3   1   1   0

Верификация полученных результатов была сделана кодом с прямым перебором всех вариантов перестановок. Для его работы надо установить пакет `Combinatorics`. Функция `FindMinima()` по заданной матрице вычисляет суммы её наддиагональных элементов при различных одновременных перестановках строк/столбцов и выводит перестановку и соответствующую сумму тогда, когда
эта сумма не превышает задаваемую необязательным параметром величину (по умолчанию -- нулевую, поскольку предполагалось применять её к «облегчённым» матрицам потерь). Функция просто содержит цикл, в котором перебираются все возможные
перестановки, формируемые вызовом функции `nthperm!()` из пакета `Combinatorics` по тождественной перестановке (которая создаётся всякий раз с помощью `Vector(1:W)`) и условному номеру перестановки. Для каждой из получаемых перестановок формируется преобразованная матрица и подсчитывается сумма её наддиагональных элементов. Если сумма не превышает `MinVal`, перестановка и сумма выводятся. Понятно, что задавать величину `MinVal` имеет смысл как можно более близкой к предполагаемому минимальному значению, иначе вывод будет слишком длинным (и долгим). Предельный размер матриц, обрабатываемых функцией, ограничен максимально возможной величиной факториала для функции `factorial()` и равен $20 \times 20$.

In [21]:
using Combinatorics

function FindMinima(ScoreMatrix::Matrix{Int}, MinVal::Int=0)
    (H,W) = size(ScoreMatrix)
    @assert W == H
    TestLength = factorial(W)
    for K = 1:TestLength
        Permutation = Vector(1:W)
        nthperm!(Permutation,K)
        USum = UpperSum(ReOrder(ScoreMatrix,Permutation))
        USum <= MinVal && println("$Permutation : $USum")
    end
end

FindMinima (generic function with 2 methods)

Вызов этой функции для «облегчённой» матрицы потерь `LMT` показывает наличие трёх минимальных значений, отличающихся циклической перестановкой трёх начальных индексов.

In [22]:
FindMinima(LMT,-75)

[1, 4, 7, 3, 8, 6, 5, 10, 9, 2] : -75
[1, 7, 4, 3, 6, 8, 5, 10, 9, 2] : -75
[1, 7, 4, 3, 8, 5, 6, 10, 9, 2] : -75
[1, 7, 4, 3, 8, 6, 5, 9, 10, 2] : -75
[1, 7, 4, 3, 8, 6, 5, 10, 2, 9] : -75
[1, 7, 4, 3, 8, 6, 5, 10, 9, 2] : -77
[1, 7, 4, 3, 8, 6, 10, 5, 9, 2] : -75
[1, 7, 4, 8, 3, 6, 5, 10, 9, 2] : -75
[4, 1, 3, 7, 8, 6, 5, 10, 9, 2] : -75
[4, 1, 7, 3, 6, 8, 5, 10, 9, 2] : -75
[4, 1, 7, 3, 8, 5, 6, 10, 9, 2] : -75
[4, 1, 7, 3, 8, 6, 5, 9, 10, 2] : -75
[4, 1, 7, 3, 8, 6, 5, 10, 2, 9] : -75
[4, 1, 7, 3, 8, 6, 5, 10, 9, 2] : -77
[4, 1, 7, 3, 8, 6, 10, 5, 9, 2] : -75
[4, 1, 7, 8, 3, 6, 5, 10, 9, 2] : -75
[4, 7, 1, 3, 8, 6, 5, 10, 9, 2] : -75
[7, 1, 4, 3, 8, 6, 5, 10, 9, 2] : -75
[7, 4, 1, 3, 6, 8, 5, 10, 9, 2] : -75
[7, 4, 1, 3, 8, 5, 6, 10, 9, 2] : -75
[7, 4, 1, 3, 8, 6, 5, 9, 10, 2] : -75
[7, 4, 1, 3, 8, 6, 5, 10, 2, 9] : -75
[7, 4, 1, 3, 8, 6, 5, 10, 9, 2] : -77
[7, 4, 1, 3, 8, 6, 10, 5, 9, 2] : -75
[7, 4, 1, 8, 3, 6, 5, 10, 9, 2] : -75


Таким образом, два из трёх минимальных значений рекурсивным вариантом эвристического алгоритма не обнаруживаются.

## Пример 4

In [23]:
LBW245 = [0 -3 1 -1 -1 -3 1; 3 0 -1 -1 1 -1 -1; -1 1 0 -1 1 -1 5; 1 1 1 0 -3 -5 -1; 1 -1 -1 3 0 -5 1; 3 1 1 5 5 0 5; -1 1 -5 1 -1 -5 0]

7×7 Array{Int32,2}:
  0  -3   1  -1  -1  -3   1
  3   0  -1  -1   1  -1  -1
 -1   1   0  -1   1  -1   5
  1   1   1   0  -3  -5  -1
  1  -1  -1   3   0  -5   1
  3   1   1   5   5   0   5
 -1   1  -5   1  -1  -5   0

Эвристический алгоритм Литвака (как и его рекурсивный вариант) здесь не работает -- по той простой причине, что анализ строки с минимальной суммой сразу «уводит» поиск в «другом» направлении: «лидирующей» строкой становится седьмая, а не первая. Получаемое ранжирование (в смысле расстояния от истинного минимума) весьма близко, но не преобразуется функцией «оптимизации» далее и остаётся неизменным по причине излишней близости к значению истинного минимума, а «лишние» минус единицы в «облегчённой» матрице потерь, одну из которых было бы хорошо обменять на единицу, стоят далеко от главной диагонали и не могут подвергнуться «изолированной» перестановке в процессе «оптимизации».

In [24]:
print(HeuristicIterations(LBW245))

[7, 1, 4, 5, 2, 3, 6]

In [25]:
print(OptimizeRanking!([7,1,4,5,2,3,6],LBW245))

[7, 1, 4, 5, 2, 3, 6]

In [26]:
R = copy(LBW245); print(HeuristicIterations!(R))

[7, 1, 4, 5, 2, 3, 6]

In [27]:
ReOrder(LBW245,[7, 1, 4, 5, 2, 3, 6])

7×7 Array{Int32,2}:
  0  -1   1  -1   1  -5  -5
  1   0  -1  -1  -3   1  -3
 -1   1   0  -3   1   1  -5
  1   1   3   0  -1  -1  -5
 -1   3  -1   1   0  -1  -1
  5  -1  -1   1   1   0  -1
  5   3   5   5   1   1   0

In [28]:
RecursiveIteration!(LBW245)

[7, 1, 4, 5, 2, 3, 6] : -33



Таким образом, все функции: `Heuristicterations()`, `Heuristicterations!()`, `RecursiveIteration!()` -- возвращают ранжирование `7,1,4,5,2,3,6`, причём его «оптимизация» не приводит к другому ранжированию. Однако, легко вычислить сумму наддиагональных элементов преобразованной с его помощью матрицы потерь, она будет равна -33. При этом ранжирование `1,2,4,7,5,3,6` даёт сумму наддиагональных элементов -35!

Истинное оптимальное (и не найденное) ранжирование:

In [29]:
UpperSum(ReOrder(LBW245,[1,2,4,7,5,3,6]))

-35

In [32]:
ReOrder(LBW245,[1,2,4,7,5,3,6])

7×7 Array{Int32,2}:
  0  -3  -1   1  -1   1  -3
  3   0  -1  -1   1  -1  -1
  1   1   0  -1  -3   1  -5
 -1   1   1   0  -1  -5  -5
  1  -1   3   1   0  -1  -5
 -1   1  -1   5   1   0  -1
  3   1   5   5   5   1   0

In [31]:
FindMinima(LBW245,-33)

[1, 2, 4, 5, 7, 3, 6] : -33
[1, 2, 4, 7, 3, 5, 6] : -33
[1, 2, 4, 7, 5, 3, 6] : -35
[1, 2, 4, 7, 5, 6, 3] : -33
[1, 2, 7, 3, 4, 5, 6] : -33
[1, 2, 7, 4, 5, 3, 6] : -33
[1, 4, 2, 7, 5, 3, 6] : -33
[1, 4, 5, 2, 7, 3, 6] : -33
[1, 4, 7, 5, 2, 3, 6] : -33
[4, 7, 1, 5, 2, 3, 6] : -33
[7, 1, 2, 3, 4, 5, 6] : -33
[7, 1, 2, 4, 5, 3, 6] : -33
[7, 1, 4, 5, 2, 3, 6] : -33
[7, 3, 1, 2, 4, 5, 6] : -33
[7, 3, 1, 4, 5, 2, 6] : -33
